In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, namedtuple
from scipy.stats import multivariate_normal
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import scale
import statsmodels.api as sm


Result = namedtuple("Result", ("predicted", "actual"))
TP = Result(1, 1)
FP = Result(1, 0)
TN = Result(0, 0)
FN = Result(0, 1)

## Problem 1

In [ ]:
data = pd.read_csv("admission-1.csv").query("De == 'admit' | De == 'notadmit'")
# `sklearn.naive_bayes.CategoricalNB` does not support negative value (part c)
data[data.loc[:, "De"] == "admit"] = 1
data[data.loc[:, "De"] == "notadmit"] = 0
data.loc[:, "De"] = data.loc[:, "De"].astype("int")

# annotation for python 3.9 and above
# for 3.8 and under, please use `typing.List` and `typing.Dict`
cv_data: list[dict[str, pd.DataFrame]] = []
for itrain, itest in KFold(shuffle=True).split(data):
    cv_data.append(
        {
            "train_x": data.loc[itrain, ["GPA", "GMAT"]],
            "train_cls": data.loc[itrain, "De"],
            "test_x":data.loc[itest, ["GPA", "GMAT"]],
            "test_cls": data.loc[itest, "De"]
        }
    )


def estimate(index: int, estimator, epilog: str = None) -> None:
    train_x: pd.DataFrame = cv_data[index]["train_x"]
    train_cls: pd.DataFrame = cv_data[index]["train_cls"]
    test_x: pd.DataFrame = cv_data[index]["test_x"]
    test_cls: pd.DataFrame = cv_data[index]["test_cls"]

    estimator.fit(train_x, train_cls)
    predicted_cls: np.ndarray = estimator.predict(test_x)

    counter: Counter[tuple[str, str], int] = Counter(
        (Result(i, j) for i, j in zip(predicted_cls, test_cls))
    )

    matrix = pd.DataFrame(
        index=["Predicted Admit", "Predicted NotAdmit"],
        columns=["Actual Admit", "Actual NotAdmit"]
    )
    matrix.iloc[0, 0] = counter[TP]
    matrix.iloc[0, 1] = counter[FP]
    matrix.iloc[1, 0] = counter[FN]
    matrix.iloc[1, 1] = counter[TN]

    accuracy: float = (counter[TP] + counter[TN]) / counter.total()

    to_print = [
        f"Iteration {index + 1}",
        f"Confusion Matrix:\n\n{matrix}",
        f"Accuracy: {accuracy}"
    ]

    if epilog is not None:
        to_print.append(epilog)

    print(
        *to_print,
        sep = "\n\n",
        end="\n\n\n"
    )

### a

In [ ]:
def bayes(index: int) -> None:
    estimator = GaussianNB()
    estimate(index, estimator)


for i in range(len(cv_data)):
    bayes(i)

### b

In [ ]:
K_VALUES = (1, 3, 5)


def knn(index: int, k: int) -> None:
    estimator = KNeighborsClassifier(n_neighbors=k)
    estimate(index, estimator, epilog=f"k: {k}")


for i in range(len(cv_data)):
    for k in K_VALUES:
        knn(i, k)

### c

In [ ]:
data = pd.concat(
    (
        pd.DataFrame(
            scale(data.loc[:, ["GPA", "GMAT"]]),
            columns=["GPA", "GMAT"]
        ),
        data.iloc[:, -1]
    ),
    axis=1
)

cv_data: list[dict[str, pd.DataFrame]] = []
for itrain, itest in KFold(shuffle=True).split(data):
    cv_data.append(
        {
            "train_x": data.loc[itrain, ["GPA", "GMAT"]],
            "train_cls": data.loc[itrain, "De"],
            "test_x":data.loc[itest, ["GPA", "GMAT"]],
            "test_cls": data.loc[itest, "De"]
        }
    )

#### bayes

In [ ]:
for i in range(len(cv_data)):
    bayes(i)

#### knn

In [ ]:
K_VALUES = (1, 3, 5)

for i in range(len(cv_data)):
    for k in K_VALUES:
        knn(i, k)

## Problem 2

In [ ]:
data = pd.read_csv("RidingMowers.csv")

data_own = data[data["Ownership"]=="Owner"].iloc[:, :2]
data_non = data[data["Ownership"]=="Nonowner"].iloc[:, :2]

# prior probablity
prior_own = data_own.shape[0] / data.shape[0]
prior_non = data_non.shape[0] / data.shape[0]

# mean
mean_own = data_own.mean()
mean_non = data_non.mean()

# covariance matrix
cov_own = data_own.cov()
cov_non = data_non.cov()

for i in data.index:
    x_own = multivariate_normal.pdf(
        (data.iloc[i, 0], data.iloc[i, 1]),
        mean_own,
        np.array(cov_own)
    )
    x_non = multivariate_normal.pdf(
        (data.iloc[i, 0], data.iloc[i, 1]),
        mean_non,
        np.array(cov_non)
    )
    post_own = prior_own * x_own
    post_non = prior_non * x_non
    data.loc[i, "Prob_Own"] = post_own / (post_own + post_non)
    data.loc[i, "Prob_Non"] = post_non / (post_own + post_non)

data

### AUC

In [ ]:
CUT_OFF = (0, 0.25, 0.5, 0.75, 1)


def foo(cut_off: int) -> tuple[int, int]:
    counter = Counter()

    for i, j in enumerate(data.loc[:, "Prob_Own"]):
        if j >= cut_off:
            if data.loc[i, "Ownership"] == "Owner":
                counter[TP] += 1
            else:
                counter[FP] += 1
        else:
            if data.loc[i, "Ownership"] == "Owner":
                counter[FN] += 1
            else:
                counter[TN] += 1

    sensitivity = counter[TP] / (counter[TP] + counter[FN])
    _1_specificity = 1 - (counter[TN] / (counter[TN] + counter[FP]))

    return _1_specificity, sensitivity


plt.plot(*zip(*[foo(i) for i in CUT_OFF]))

## Problem 3

In [ ]:
COLUMNS = [
    "CRIM", "ZN", "INDUS", "NOX", "RM",
    "AGE", "DIS", "TAX", "PTRATIO", "LSTAT"
]

data = pd.read_csv("BostonHousing-2.csv")
data[COLUMNS] = (data[COLUMNS] - data[COLUMNS].mean()) / data[COLUMNS].std()

### a

In [ ]:
train_df = data.sample(frac=0.6)
test_df = data.drop(train_df.index)

# isinstance([*train_df.itertuples()][0], tuple)

train_x = [i[1:-2] for i in train_df.itertuples()]
train_cls = [*train_df["MEDV"]]
test_x = [i[1:-2] for i in test_df.itertuples()]
test_cls = [*test_df["MEDV"]]

for k in range(1, 6):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(train_x, train_cls)
    predicted_cls = knn.predict(test_x)
    RMSE = np.sqrt(metrics.mean_squared_error(test_cls, predicted_cls))
    print(f"RMSE for k = {k} is {RMSE}")

Because RMSE for k equal to specific value varies significantly as argument `random_state` in `pandas.DataFrame.sample` method changes, we cannot determine which k value has better performance.

### b

In [ ]:
new = pd.DataFrame(
    [[0.2, 0, 7, 0, 0.538, 6, 62, 4.7, 4, 307, 21, 10]],
    columns=[
        'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
        'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT'
    ]
)
new[COLUMNS] = (new[COLUMNS] - data[COLUMNS].mean()) / data[COLUMNS].std()
new_x = [i[1:] for i in new.itertuples()]

# as we discussed in part a, best k cannot be determined
# we hereby assume best k is equal to 3,
# which came from my teammate Junxiang Yang
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(train_x, train_cls)
predicted_cls = knn.predict(new_x)

print(f"MEDV is predicted to be {predicted_cls[0]}")

### c

In [ ]:
predicted_cls = knn.predict(train_x)
RMSE = np.sqrt(metrics.mean_squared_error(train_cls, predicted_cls))

print(f"RMSE for training data is {RMSE}")

### d

- sample size is too small to fit the model.

- new data has anomaly value.

### e

It has a heavy computational burden when comes to a large dataset, because its time complexity is $O(n^2)$.

Opreations are as follow:

1. compute distantce from data to predict to all data points exist;

1. find k combinations with shortest distance;

1. find best k value by performance evaluation.

## Problem 4

### a

$$
\begin{align*}
estimated\ profit &= \$\ 2500 \times 1000\\
&= \$\ 2500000
\end{align*}
$$

### b

The first 10%.

### c

Cut-off value $\$ 2500$ is equal to the sale effort, which means $1.00$ lift. According to the chart, we should proceed dwon top 500 customers (50%).

### d

A twp-stage process instead of the later option can be applied to variable new leads and reduce unnecessary commercial cost from people who do not interest in the product, which means greater profit.

## Peoblem 5

### 1

- Training set will be used for training model.

- Validation set will be used for obtain the best hyperparameter combination in order to optimize the performance of the model.

### 2

In [ ]:
data = pd.read_csv("BostonHousing-2.csv")
data_x = [i[1:] for i in data[["CRIM", "CHAS", "RM"]].itertuples()]
data_y = [*data["MEDV"]]

lr = LinearRegression()
lr.fit(data_x, data_y)

print(f"coefficients: {lr.coef_}\n\ny-intercept: {lr.intercept_}")

### 3

In [ ]:
predicted = lr.predict([(0.1, 0, 6)])
print(f"MEDV is {predicted[0]}")

### 4

In [ ]:
sns.heatmap(data.corr())

#### 1

`INDUS`, `TAX` and `NOX` have strong positive correlations to each other, therefore they are likely to be measuring the same thing.

#### 2

In [ ]:
data.corr()

Remove one of `RAD` and `TAX`.

#### 3

##### Backward Elimination

In [ ]:
data_x = [(*i[1:9], *i[11:13]) for i in data.itertuples()]
sm.OLS(data_y, data_x).fit().summary()

In [ ]:
# drop `AGE`
data_x = [(*i[:6], *i[7:]) for i in data_x]
sm.OLS(data_y, data_x).fit().summary()

In [ ]:
# drop `NOX`
data_x = [(*i[:4], *i[5:]) for i in data_x]
sm.OLS(data_y, data_x).fit().summary()
# all variables below significant value

In [ ]:
lr = LinearRegression()
lr.fit(data_x, data_y)

data_y = np.array(data_y)
predicted = lr.predict(data_x)

RMSE_back = np.sqrt(metrics.mean_squared_error(data_y, predicted))
MAPE_back = np.mean(np.abs((data_y - predicted) / data_y)) * 100
error_back = np.abs(predicted - data_y)
Mean_error_back = np.mean(error_back)

##### Forward Selection

In [ ]:
bar = lambda: data.itertuples()

sm.OLS(
    data_y,
    [(i.CRIM, ) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.RM) for i in bar()]
).fit().summary()
# `ZN` exceeds significant value, drop `RM`

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.AGE) for i in bar()]
).fit().summary()
# `RM` exceeds significant value, drop `AGE`

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.DIS) for i in bar()]
).fit().summary()
# all p values below 0.05

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.DIS, i.PTRATIO) for i in bar()]
).fit().summary()
# `RM` exceeds significant value, drop `PTRATIO`

sm.OLS(
    data_y,
    [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.DIS, i.LSTAT) for i in bar()]
).fit().summary()
# all p values below 0.05

In [ ]:
data_x = [(i.CRIM, i.ZN, i.INDUS, i.CHAS, i.NOX, i.DIS, i.LSTAT) for i in bar()]

lr = LinearRegression()
lr.fit(data_x, data_y)

predicted = lr.predict(data_x)

RMSE_fore = np.sqrt(metrics.mean_squared_error(data_y, predicted))
MAPE_fore = np.mean(np.abs(data_y - predicted) / data_y) * 100
error_fore = np.abs(predicted - data_y)
Mean_error_fore = np.mean(error_fore)

##### Stepwise Selection

In [ ]:
# `ZN` is the first variable that exceeds significant value, drop `ZN`

# add `AGE`
sm.OLS(
    data_y,
    [(i.CRIM, i.INDUS, i.CHAS, i.NOX, i.RM, i.AGE) for i in bar()]
).fit().summary()
# `AGE` exceeds, drop

# add `DIS`
sm.OLS(
    data_y,
    [(i.CRIM, i.INDUS, i.CHAS, i.NOX, i.RM, i.DIS) for i in bar()]
).fit().summary()
# all p values below 0.05

# add `PTRATIO`
sm.OLS(
    data_y,
    [(i.CRIM, i.INDUS, i.CHAS, i.NOX, i.RM, i.DIS, i.PTRATIO) for i in bar()]
).fit().summary()
# all p values below 0.05

# add `LSTAT`
sm.OLS(
    data_y,
    [(i.CRIM, i.INDUS, i.CHAS, i.NOX, i.RM, i.DIS, i.PTRATIO, i.LSTAT) for i in bar()]
).fit().summary()
# `NOX` exceeds, drop

sm.OLS(
    data_y,
    [(i.CRIM, i.INDUS, i.CHAS, i.RM, i.DIS, i.PTRATIO, i.LSTAT) for i in bar()]
).fit().summary()
# all p values below 0.05

In [ ]:
data_x = [(i.CRIM, i.INDUS, i.CHAS, i.RM, i.DIS, i.PTRATIO, i.LSTAT) for i in bar()]

lr = LinearRegression()
lr.fit(data_x, data_y)

predicted = lr.predict(data_x)

RMSE_step = np.sqrt(metrics.mean_squared_error(data_y, predicted))
MAPE_step = np.mean(np.abs((data_y - predicted) / data_y)) * 100
error_step = np.abs(predicted - data_y)
Mean_error_step = np.mean(error_step)

##### Result

In [ ]:
fig, ax = plt.subplots(1, 3)

ax[0].hist(error_back)
ax[1].hist(error_fore)
ax[2].hist(error_step)

In [ ]:
pd.DataFrame(
    [
        [RMSE_back, MAPE_back, Mean_error_back],
        [RMSE_fore, MAPE_fore, Mean_error_fore], 
        [RMSE_step, MAPE_step, Mean_error_step]
    ],
    index=["Backward", "Fordward", "Stepwise"],
    columns=["RMSE", "MAPE", "Mean Error"]
)